In [1]:
from pyspark.sql import SparkSession
from delta import *
from etl.extract_data import extract_data
from etl.transform_data import clean_data, format_column_name
from etl.load_data import load_data


Set up Spark + Delta Lake

In [2]:
# Setup Spark + Delta Lake ( define DeltaSparkSessionExtension and DeltaCatalog)
builder = SparkSession.builder.appName("DeltaLake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
print("Spark + Delta Lake setup is successful!")
print(spark)

Spark + Delta Lake setup is successful!


Extract and Transform data from a file

In [6]:
# Extract data from a file .csv
df = extract_data(spark, "data/tourism_dataset.csv")
df_clean = clean_data(df)
df_formatted = format_column_name(df_clean)
new_df = df_formatted
try: 
    new_df.show(5)
except Exception as e:
    print(e)

+----------+---+--------------------+-----------------------+-------------+------------+--------------------+-------------+--------+--------------+--------------------+-----------------------+---------------------+------------+
|Tourist_ID|Age|           Interests|Preferred_Tour_Duration|Accessibility|   Site_Name|       Sites_Visited|Tour_Duration|Route_ID|Tourist_Rating|System_Response_Time|Recommendation_Accuracy|VR_Experience_Quality|Satisfaction|
+----------+---+--------------------+-----------------------+-------------+------------+--------------------+-------------+--------+--------------+--------------------+-----------------------+---------------------+------------+
|         1| 48|['Architecture', ...|                      5|        False|Eiffel Tower|['Eiffel Tower', ...|            7|    1000|           1.6|                3.73|                     97|                  4.5|           3|
|         2| 37|['Cultural', 'Nat...|                      6|        False|   Colosseum|

Save data to a Delta table


In [7]:
# Save data to a Delta table
load_data(new_df, "data/delta_table")

In [10]:
# Read data from a Delta table
try:
    df_delta = spark.read.format("delta").load("data/delta_table")
    df_delta.show(5)
except Exception as e:
    print(e)


An error occurred while calling o93.load.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at io.delta.storage.HadoopFileSystemLogStore.listFrom(HadoopFileSystemLogStore.java:59)
	at org.apache.spark.sql.delta.storage.LogStoreAdaptor.listFrom(LogStore.scala:452)
	at org.apache.spark.sql.delta.storage.DelegatingLogStore.listFrom(DelegatingLogStore